In [1]:
import sys
from pathlib import Path

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")
    
import pandas as pd
import numpy as np
import ast

import tasks.run_helper

In [2]:
df = pd.read_csv(
    "../data/100_annotators.csv",
    converters={"annot_personality_characteristics": ast.literal_eval},
)
df["toxicity"] = df.annotation.apply(lambda x: x[-1]).astype(int)
df["annot_politics"] = df.annot_personality_characteristics.apply(
    lambda x: x[0]
)
df.annot_age = pd.cut(df.annot_age, bins=4)
df.message_id = df.message_id.astype(str)
df["comment_key"] = df.conv_id + df.message_id

df = df.loc[
    :,
    [
        "conv_id",
        "message_id",
        "comment_key",
        "message",
        "toxicity",
        "annot_age",
        "annot_sex",
        "annot_sexual_orientation",
        "annot_demographic_group",
        "annot_current_employment",
        "annot_education_level",
        "annot_politics",
    ],
]
df = df.groupby(["conv_id", "message_id", "comment_key", "message"]).apply(
    lambda x: pd.Series(
        {
            col: x[col].tolist()
            for col in df.columns
            if col not in ["conv_id", "message_id", "comment_key", "message"]
        }
    ),
    include_groups=False,
)
df["fake_index"] = 1
df["random"] = tasks.preprocessing.get_rand_col(df, "annot_sex")


In [3]:
sdb_columns = [
    "annot_age",
    "annot_sex",
    "annot_demographic_group",
    "annot_sexual_orientation",
    "annot_current_employment",
    "annot_education_level",
    "annot_politics"
]
res = tasks.run_helper.run_all_results(
    df=df,
    sdb_columns=sdb_columns,
    discussion_id_col="fake_index",
    value_col="toxicity",
    comment_key_col="comment_key",
)
res

Evaluating SDB dimensions:   0%|          | 0/7 [00:00<?, ?it/s]

1          
                                                     kappa    pvalue
sdb_column                                                          
annot_age                (14.927, 33.25]          0.084521  0.816667
                         (33.25, 51.5]            0.300373  0.816667
                         (51.5, 69.75]           -0.739851  0.816667
                         (69.75, 88.0]            0.006488  0.816667
annot_current_employment blue-collar             -0.269699  0.816667
                         unemployed               0.656213  0.816667
                         white-collar            -0.376642  0.816667
annot_demographic_group  asian                    1.508272  0.816667
                         black                    0.495127  0.816667
                         other                   -1.807158  0.816667
                         white                    1.036643  0.816667
annot_education_level    high-school              0.927314  0.800000
                         none                    -1.178787  0.800000
                         university               0.309057  0.800000
annot_politics           apolitical              -2.315889  0.816667
                         left-wing liberal        1.052454  0.816667
                         right-wing conservative  0.632289  0.816667
annot_sex                female                  -0.078850  0.833333
                         male                    -0.577009  0.833333
                         non-binary               5.043546  0.833333
annot_sexual_orientation bisexual                 3.034525  0.850000
                         homosexual              -1.180362  0.850000
                         other                   -4.487805  0.850000
                         straight                 0.377574  0.850000

In [4]:
tasks.run_helper.results_to_latex(
    res, output_path=Path("test.tex"), dataset_name="test"
)

Results exported to /home/dimits/Documents/research/aposteriori-unimodality/notebooks/test.tex


In [5]:
tasks.run_helper.run_result(
    df,
    discussion_id_col="fake_index",
    sdb_column="random",
    value_col="toxicity",
    comment_key_col="comment_key",
)

1          
      kappa    pvalue
6 -0.488069  0.833333
9  0.518640  0.833333
2  2.047907  0.833333
7  0.770365  0.833333
1 -1.562009  0.833333
4 -2.733315  0.833333
5 -0.503443  0.833333
3 -3.323076  0.833333
8  1.215435  0.833333